In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
deal = pd.read_csv('deal_new.csv')
deal.head(2)

In [ ]:
order = pd.read_csv('order.csv',index_col=None)
order.head(2)

In [ ]:
data = pd.merge(order,deal,how='left',on=['business_hour_guid'])
data

In [ ]:
#計算商品銷量
data['sold_unit'] = data['subtotal'] / data['item_price']

In [ ]:
#將時間由object轉換成datetime型別，並將年月轉換成數字
data['create_time'] = pd.to_datetime(data['create_time'])
data['year'] = data['create_time'].dt.year
data['month'] = data['create_time'].dt.month

In [ ]:
#篩選出2019年4月份的資料
apr_data = data[(data['year']==2019) & (data['month']==4)]

In [ ]:
#計算各商品4月份的總銷量
total_sold = apr_data.groupby(['business_hour_guid','code'],as_index=False)['sold_unit'].sum()

In [ ]:
#讀取各商品4月份的總瀏覽次數
browsing_data = pd.read_csv('count_time.csv',names=['business_hour_guid','times'])
browsing_data.head()

In [ ]:
ROI = pd.merge(total_sold,browsing_data,how='left',on=['business_hour_guid'])

In [ ]:
#讀取Google的商品分類資料
google = pd.read_excel('google_category.xls')
google.head(2)

In [ ]:
google_cat = google.rename(columns={'id':'code','level1':'category'})
google_cat.head(2)

In [ ]:
cate_ROI = pd.merge(ROI,google_cat,how='left',on=['code'])
cate_ROI.head(2)

In [ ]:
#依Google類別計算出各類別商品的平均瀏覽次數及平均銷量
result=cate_ROI.groupby(['category'],as_index=False)['times','sold_unit'].mean()

In [ ]:
#計算出各類別商品的轉換率
result['percentage']= ((result['sold_unit']/result['times'])*100).round(0)
result

In [ ]:
fig = plt.figure(figsize=(48, 36))
ax1 = sns.lmplot(data=result,x='times', y='sold_unit',hue='category')
ax1.set(xlim=(0,2000))
ax1.set(ylim=(0,350))
plt.xlabel('Browsing Times')
plt.ylabel('Sold Unit')
plt.title('Browsing History Analysis')
plt.show()

In [ ]:
ax1.savefig('lmplot.jpg',dpi=300)

In [ ]:
fig = plt.figure(figsize=(48, 36))
splot=sns.barplot(data=result,x='category',y='percentage')
plt.xlabel('Category')
plt.ylabel('Sold Percentage')
plt.title('Conversion Rate')

In [ ]:
fig.savefig('barplot.jpg',dpi=300)